In [1]:
import pandas as pd
import numpy as np
import zipfile
from IPython.display import display
import re
from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()
from collections import Counter

In [219]:
dump = False
load = False
def dump_to_pickle(obj, path):
    if dump:
        with open(path, 'wb') as pickled:
            import pickle
            pickle.dump(obj, pickled)
        print("dumped")
    else:
        print("dump suppressed")
def load_from_pickle(path):
    if load:
        with open(path, 'rb') as pickled:
            import pickle
            return pickle.load(pickled)
        print("loaded")
    else:
        raise("load suppressed")

In [3]:
from collections import defaultdict
class defaultgetdict(defaultdict):
    """like collections.defaultdict but does not insert missing key if not assigned"""
    def __missing__(self, key):
        if self.default_factory is None: raise KeyError(key)
        return self.default_factory()

In [4]:
def read_infobox(file):
    def tr(row):
        from itertools import groupby
        from operator import itemgetter
        def featurize(item, counter):
            partitioned = item.partition(':')
            field_partitioned = partitioned[0].rsplit('_', 1)
            counter[field_partitioned[0]] += 1
            field_partitioned[1] = int(field_partitioned[1])
            return partitioned[2], field_partitioned
        from collections import Counter
        field_counter = Counter()
        projection = [featurize(item, field_counter) for item in row.split('\t') if not item.endswith(':<none>')]
        for item in projection:
            item[1].append(field_counter[item[1][0]] + 1 - item[1][1])
        projection = defaultgetdict(list, [(k, list([t[1] for t in v])) for k,v in groupby(sorted(projection, key=itemgetter(0)), key=itemgetter(0))])
        return {'counter':field_counter, 'data':projection}
    import sys
    nrows = 10
    nrows = sys.maxsize
    infobox = pd.DataFrame(pd.read_csv(file, sep='\n', header=None, names=['infobox'], nrows=nrows)['infobox'].progress_apply(tr).tolist())
    return infobox

In [5]:
train_infobox = read_infobox('../wikipedia-biography-dataset/train/train.box')

A Jupyter Widget

In [6]:
test_infobox = read_infobox('../wikipedia-biography-dataset/test/test.box')

A Jupyter Widget

In [7]:
valid_infobox = read_infobox('../wikipedia-biography-dataset/valid/valid.box')

A Jupyter Widget

In [8]:
dump_to_pickle(train_infobox, 'train.box.pickle')

dump suppressed


In [9]:
dump_to_pickle(test_infobox, 'test.box.pickle')

dump suppressed


In [10]:
dump_to_pickle(valid_infobox, 'valid.box.pickle')

dump suppressed


In [12]:
train_infobox = load_from_pickle('train.box.pickle')
valid_infobox = load_from_pickle('valid.box.pickle')
test_infobox = load_from_pickle('test.box.pickle')

TypeError: exceptions must derive from BaseException

In [13]:
train_infobox.iloc[0]['data']

defaultgetdict(list,
               {'1954': [['birth_date', 1, 1]],
                'aircraft': [['occupation', 1, 4]],
                'and': [['occupation', 3, 2]],
                'designer': [['occupation', 2, 3]],
                'extra': [['name', 2, 1], ['article_title', 2, 1]],
                'german': [['nationality', 1, 1]],
                'manufacturer': [['occupation', 4, 1]],
                'walter': [['name', 1, 2], ['article_title', 1, 2]]})

In [14]:
train_infobox.iloc[0]['counter']

Counter({'article_title': 2,
         'birth_date': 1,
         'name': 2,
         'nationality': 1,
         'occupation': 4})

In [15]:
def read_nb(file):
    import sys
    nrows = 10
    nrows = sys.maxsize
    nb = pd.concat([pd.Series([0]), pd.read_csv(file, sep='\n', header=None, nrows=nrows, squeeze=True).cumsum()], ignore_index=True).iloc[:-1]
    return nb

train_nb = read_nb('../wikipedia-biography-dataset/train/train.nb')

In [16]:
valid_nb = read_nb('../wikipedia-biography-dataset/valid/valid.nb')
test_nb = read_nb('../wikipedia-biography-dataset/test/test.nb')

In [17]:
def read_sent(file, nb):
    import sys
    nrows = 10
    nrows = sys.maxsize
    sent = pd.read_csv(file, sep='\n', header=None, nrows=nrows, squeeze=True).iloc[nb]
    return sent

train_sent = read_sent('../wikipedia-biography-dataset/train/train.sent', train_nb)

In [18]:
valid_sent = read_sent('../wikipedia-biography-dataset/valid/valid.sent', valid_nb)
test_sent = read_sent('../wikipedia-biography-dataset/test/test.sent', test_nb)

In [19]:
def replace_with_copy_token(sent, infobox):
    def replace_fn(row):
        replaced = [(w, row['data'][w]) for w in row['sentence'].split()]
        return replaced
    return pd.concat([sent.reset_index(name='sentence'), infobox], axis=1).progress_apply(replace_fn, axis=1)
test_sent_tokens = replace_with_copy_token(test_sent, test_infobox)

A Jupyter Widget

In [20]:
train_sent_tokens = replace_with_copy_token(train_sent, train_infobox)
valid_sent_tokens = replace_with_copy_token(valid_sent, valid_infobox)

A Jupyter Widget

A Jupyter Widget

In [21]:
train_nb = load_from_pickle('train.nb.pickle')
valid_nb = load_from_pickle('valid.nb.pickle')
test_nb = load_from_pickle('test.nb.pickle')

TypeError: exceptions must derive from BaseException

In [ ]:
train_sent_tokens = load_from_pickle('train.sent.tokens.pickle')
valid_sent_tokens = load_from_pickle('valid.sent.tokens.pickle')
test_sent_tokens = load_from_pickle('test.sent.tokens.pickle')

In [22]:
test_sent_tokens.head()

0    [(leonard, []), (shenoff, []), (randle, [['nam...
1    [(philippe, [['caption', 1, 2], ['article_titl...
2    [(miroslav, [['name', 1, 2], ['article_title',...
3    [(john, [['fullname', 1, 2]]), (``, []), (jack...
4    [(william, [['birth_name', 1, 3]]), (ato, [['b...
dtype: object

In [23]:
def sent_token_analyzer(row):
    return [pair[0] for pair in row if not pair[1]]
def vectorize(sent_tokens):
    from sklearn.feature_extraction.text import CountVectorizer
    vectorizer = CountVectorizer(analyzer=sent_token_analyzer)
    return (vectorizer.fit_transform(sent_tokens), vectorizer)

In [24]:
train_sent_vectorized, train_sent_vectorizer = vectorize(train_sent_tokens)
valid_sent_vectorized, valid_sent_vectorizer = vectorize(valid_sent_tokens)
test_sent_vectorized, test_sent_vectorizer = vectorize(test_sent_tokens)

In [25]:
dump_to_pickle(train_sent_vectorizer, 'train.sent.vectorizer.pickle')
dump_to_pickle(train_sent_vectorized, 'train.sent.vectorized.pickle')
dump_to_pickle(valid_sent_vectorizer, 'valid.sent.vectorizer.pickle')
dump_to_pickle(valid_sent_vectorized, 'valid.sent.vectorized.pickle')
dump_to_pickle(test_sent_vectorizer, 'test.sent.vectorizer.pickle')
dump_to_pickle(test_sent_vectorized, 'test.sent.vectorized.pickle')

dump suppressed
dump suppressed
dump suppressed
dump suppressed
dump suppressed
dump suppressed


In [26]:
def word_frequency(vectorized, vectorizer):
    return pd.DataFrame({'word':vectorizer.get_feature_names(), 'count':vectorized.sum(0).tolist()[0]})

In [27]:
train_frequency = word_frequency(train_sent_vectorized, train_sent_vectorizer)
valid_frequency = word_frequency(valid_sent_vectorized, valid_sent_vectorizer)
test_frequency = word_frequency(test_sent_vectorized, test_sent_vectorizer)

In [28]:
def trim_frequency(frequency, k, column_name):
    """Trim the data to at least k, but includes rows whose value of column_name is at least equal to the top-k"""
    return frequency[frequency[column_name] >= frequency.nlargest(k, column_name)[column_name].min()]

In [29]:
def get_embedding_lookup_table(vectorizer):
    return dict(zip(vectorizer.get_feature_names(), range(len(vectorizer.get_feature_names()))))

train_embedding_lookup_table = get_embedding_lookup_table(train_sent_vectorizer)
valid_embedding_lookup_table = get_embedding_lookup_table(valid_sent_vectorizer)
test_embedding_lookup_table = get_embedding_lookup_table(test_sent_vectorizer)

In [30]:
dump_to_pickle(train_embedding_lookup_table, 'train.lookup.table.pickle')
dump_to_pickle(valid_embedding_lookup_table, 'valid.lookup.table.pickle')
dump_to_pickle(test_embedding_lookup_table, 'test.lookup.table.pickle')

dump suppressed
dump suppressed
dump suppressed


In [31]:
def counter_analyzer(counter):
    from itertools import chain, repeat
    return list(chain.from_iterable(repeat(k,v)) for k, v in counter.items())
def counter_sum(counters):
    from sklearn.feature_extraction import DictVectorizer
    vectorizer = DictVectorizer()
    return (dict(zip(*reversed([vectorizer.fit_transform(tqdm_notebook(counters)).sum(0).tolist()[0], vectorizer.get_feature_names()]))), vectorizer)

In [32]:
train_infobox_sum, train_infobox_sum_vectorizer = counter_sum(train_infobox['counter'])
valid_infobox_sum, valid_infobox_sum_vectorizer = counter_sum(valid_infobox['counter'])
test_infobox_sum, test_infobox_sum_vectorizer = counter_sum(test_infobox['counter'])

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

In [33]:
dump_to_pickle(train_sent_tokens, 'train.infobox.sum.pickle')
dump_to_pickle(valid_sent_tokens, 'valid.infobox.sum.pickle')
dump_to_pickle(test_infobox_sum, 'test.infobox.sum.pickle')

dump suppressed
dump suppressed
dump suppressed


In [34]:
train_infobox_sum = load_from_pickle('train.infobox.sum.pickle')
valid_infobox_sum = load_from_pickle('valid.infobox.sum.pickle')
test_infobox_sum = load_from_pickle('test.infobox.sum.pickle')

TypeError: exceptions must derive from BaseException

In [35]:
train_infobox_lookup_table = get_embedding_lookup_table(train_infobox_sum_vectorizer)
valid_infobox_lookup_table = get_embedding_lookup_table(valid_infobox_sum_vectorizer)
test_infobox_lookup_table = get_embedding_lookup_table(test_infobox_sum_vectorizer)

In [36]:
dump_to_pickle(train_infobox_lookup_table, 'train.infobox.lookup.pickle')
dump_to_pickle(valid_infobox_lookup_table, 'valid.infobox.lookup.pickle')
dump_to_pickle(test_infobox_lookup_table, 'test.infobox.lookup.pickle')

dump suppressed
dump suppressed
dump suppressed


In [37]:
[(len(train_infobox_sum), len([{k: v} for k, v in train_infobox_sum.items() if v >= 100])),
 (len(valid_infobox_sum), len([{k: v} for k, v in valid_infobox_sum.items() if v >= 100])),
 (len(test_infobox_sum), len([{k: v} for k, v in test_infobox_sum.items() if v >= 100]))]

[(6328, 1485), (2873, 751), (2896, 746)]

In [38]:
dict(valid_infobox['counter'][0])

{'article_title': 5,
 'birth_place': 1,
 'buried': 6,
 'death_date': 3,
 'ended': 3,
 'enthroned': 3,
 'feast_day': 10,
 'name': 4,
 'nationality': 1,
 'predecessor': 2,
 'religion': 3,
 'residence': 4,
 'successor': 2,
 'title': 12,
 'type': 1}

In [131]:
train_infobox_lookup_table

{'': 0,
 '&lt;': 1,
 "'''boyfriend'''": 2,
 "'''death_date": 3,
 "'''name'''": 4,
 "''illustrations''": 5,
 "''influences&quot;": 6,
 "''novels''": 7,
 "'monarch'": 8,
 '.': 9,
 '.|m|abbr': 10,
 '_date': 11,
 '_image_size': 12,
 '_members': 13,
 '_place': 14,
 'a.k.a': 15,
 'a.k.a.': 16,
 'aarti': 17,
 'abandoned': 18,
 'abbot': 19,
 'abbrev': 20,
 'abbreviation': 21,
 'abdication_date': 22,
 'abode': 23,
 'abolished': 24,
 'above': 25,
 'abovestyle': 26,
 'abv': 27,
 'acadamyawards': 28,
 'academic': 29,
 'academic_adivsor': 30,
 'academic_adviors': 31,
 'academic_advisor': 32,
 'academic_advisors': 33,
 'academic_degree': 34,
 'academic_status': 35,
 'academy': 36,
 'academy_awards': 37,
 'academyawards': 38,
 'accessdate': 39,
 'accession': 40,
 'accn': 41,
 'accomplishment': 42,
 'accomplishments': 43,
 'account': 44,
 'accused': 45,
 'accw': 46,
 'achievement': 47,
 'achievements': 48,
 'achievements|awards': 49,
 'acronym': 50,
 'actend': 51,
 'actiev': 52,
 'active': 53,
 'activ

In [47]:
[k for k, v in globals().items() if k.startswith('train_')]

['train_infobox',
 'train_nb',
 'train_sent',
 'train_sent_tokens',
 'train_sent_vectorized',
 'train_sent_vectorizer',
 'train_frequency',
 'train_embedding_lookup_table',
 'train_infobox_sum',
 'train_infobox_sum_vectorizer',
 'train_infobox_lookup_table']

In [55]:
train_token_lengths = train_sent_tokens.progress_apply(len)
valid_token_lengths = valid_sent_tokens.progress_apply(len)
test_token_lengths = test_sent_tokens.progress_apply(len)

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

## Statistics

### Sentence

In [58]:
np.reshape(np.percentile(train_token_lengths, range(100)), [10, 10])

array([[ 1.,  8., 10., 11., 12., 13., 13., 14., 14., 14.],
       [15., 15., 15., 16., 16., 16., 16., 17., 17., 17.],
       [17., 18., 18., 18., 18., 18., 19., 19., 19., 19.],
       [20., 20., 20., 20., 21., 21., 21., 21., 21., 22.],
       [22., 22., 22., 22., 23., 23., 23., 23., 24., 24.],
       [24., 24., 25., 25., 25., 25., 26., 26., 26., 26.],
       [27., 27., 27., 27., 28., 28., 28., 29., 29., 29.],
       [30., 30., 30., 31., 31., 31., 32., 32., 32., 33.],
       [33., 34., 34., 35., 35., 36., 37., 37., 38., 39.],
       [40., 41., 42., 43., 45., 46., 48., 51., 55., 62.]])

In [80]:
np.percentile(train_token_lengths, [5, 95])

array([13., 46.])

In [82]:
np.mean(train_token_lengths)

26.05811289278978

In [59]:
max(train_token_lengths)

271

In [63]:
train_sent_tokens[0]

[('walter', [['name', 1, 2], ['article_title', 1, 2]]),
 ('extra', [['name', 2, 1], ['article_title', 2, 1]]),
 ('is', []),
 ('a', []),
 ('german', [['nationality', 1, 1]]),
 ('award-winning', []),
 ('aerobatic', []),
 ('pilot', []),
 (',', []),
 ('chief', []),
 ('aircraft', [['occupation', 1, 4]]),
 ('designer', [['occupation', 2, 3]]),
 ('and', [['occupation', 3, 2]]),
 ('founder', []),
 ('of', []),
 ('extra', [['name', 2, 1], ['article_title', 2, 1]]),
 ('flugzeugbau', []),
 ('-lrb-', []),
 ('extra', [['name', 2, 1], ['article_title', 2, 1]]),
 ('aircraft', [['occupation', 1, 4]]),
 ('construction', []),
 ('-rrb-', []),
 (',', []),
 ('a', []),
 ('manufacturer', [['occupation', 4, 1]]),
 ('of', []),
 ('aerobatic', []),
 ('aircraft', [['occupation', 1, 4]]),
 ('.', [])]

In [65]:
train_table_token_lengths = train_sent_tokens.progress_apply(lambda x: len([i for i in x if i[1]]))
valid_table_token_lengths = valid_sent_tokens.progress_apply(lambda x: len([i for i in x if i[1]]))
test_table_token_lengths = test_sent_tokens.progress_apply(lambda x: len([i for i in x if i[1]]))

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

In [66]:
np.reshape(np.percentile(train_table_token_lengths, range(100)), [10, 10])

array([[ 0.,  2.,  3.,  3.,  3.,  4.,  4.,  4.,  5.,  5.],
       [ 5.,  5.,  6.,  6.,  6.,  6.,  6.,  6.,  7.,  7.],
       [ 7.,  7.,  7.,  7.,  7.,  8.,  8.,  8.,  8.,  8.],
       [ 8.,  8.,  8.,  9.,  9.,  9.,  9.,  9.,  9.,  9.],
       [ 9.,  9., 10., 10., 10., 10., 10., 10., 10., 10.],
       [11., 11., 11., 11., 11., 11., 11., 12., 12., 12.],
       [12., 12., 12., 12., 13., 13., 13., 13., 13., 13.],
       [14., 14., 14., 14., 14., 14., 15., 15., 15., 15.],
       [16., 16., 16., 16., 17., 17., 17., 18., 18., 19.],
       [19., 20., 20., 21., 22., 23., 24., 25., 27., 31.]])

In [83]:
np.percentile(train_table_token_lengths, [5, 50, 95])

array([ 4., 11., 23.])

In [84]:
np.mean(train_table_token_lengths)

11.570467460384204

In [67]:
max(train_table_token_lengths)

124

### Infobox

In [95]:
train_infobox['data']

0         {'1954': [['birth_date', 1, 1]], 'aircraft': [...
1         {',': [['birth_place', 2, 5], ['birth_place', ...
2         {''s': [['known_for', 2, 4]], ',': [['birth_pl...
3         {',': [['birth_place', 2, 4], ['birth_place', ...
4         {''': [['years', 11, 1]], '''': [['years', 7, ...
5         {',': [['birth_place', 2, 4], ['birth_place', ...
6         {',': [['birth_place', 2, 2]], '-': [['years',...
7         {',': [['birth_place', 2, 2]], '--': [['youthy...
8         {'''': [['occupation', 6, 4], ['occupation', 8...
9         {',': [['birth_date', 3, 2], ['birth_place', 2...
10        {',': [['birth_place', 2, 2]], 'l.': [['captio...
11        {'-': [['fivefor', 1, 2], ['tenfor', 1, 2]], '...
12        {',': [['image', 3, 5], ['term_start', 3, 22],...
13        {',': [['birth_place', 3, 2], ['death_place', ...
14        {',': [['birth_place', 2, 2], ['pcupdate', 2, ...
15        {',': [['birth_place', 2, 2], ['pcupdate', 2, ...
16        {',': [['birth_place', 2, 4], 

In [88]:
train_infobox_field_count = train_infobox['data'].progress_apply(len)
valid_infobox_field_count = valid_infobox['data'].progress_apply(len)
test_infobox_field_count = test_infobox['data'].progress_apply(len)

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

In [90]:
train_infobox_token_count = train_infobox['data'].progress_apply(lambda row: sum([len(i) for i in row.values()]))
valid_infobox_token_count = valid_infobox['data'].progress_apply(lambda row: sum([len(i) for i in row.values()]))
test_infobox_token_count = test_infobox['data'].progress_apply(lambda row: sum([len(i) for i in row.values()]))

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

In [125]:
np.percentile(train_infobox_field_count, [5, 50, 95])

array([ 5., 12., 21.])

In [126]:
np.mean(train_infobox_field_count)

12.449932121532491

In [93]:
np.percentile(train_infobox_token_count, [5, 50, 95])

array([ 13.,  39., 101.])

In [94]:
np.mean(train_infobox_token_count)

45.813290449473875

In [107]:
train_infobox['data'][0].values()

dict_values([[['birth_date', 1, 1]], [['occupation', 1, 4]], [['occupation', 3, 2]], [['occupation', 2, 3]], [['name', 2, 1], ['article_title', 2, 1]], [['nationality', 1, 1]], [['occupation', 4, 1]], [['name', 1, 2], ['article_title', 1, 2]]])

In [124]:
from itertools import chain
train_infobox_field_count = train_infobox['data'].progress_apply(lambda row: len(set([i[0] for i in list(chain.from_iterable(row.values()))])))
valid_infobox_field_count = valid_infobox['data'].progress_apply(lambda row: len(set([i[0] for i in list(chain.from_iterable(row.values()))])))
test_infobox_field_count = test_infobox['data'].progress_apply(lambda row: len(set([i[0] for i in list(chain.from_iterable(row.values()))])))

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

### Custom class

In [151]:
class TupleVectorizer:
    """TODO use actual sklearn mixin"""
    def __init__(self):
        self.size = 0
        self._map = dict()
        self._features = []
    def progress_fit(self, X):
        self._map = dict()
        for x in tqdm_notebook(X):
            self.partial_fit(x)
    def partial_fit(self, x):
        if x not in self._map:
            self._map[x] = self.size
            self._features.append(x)
            self.size += 1
    def transform(self, X):
        return [self._map.get(x, -1) for x in X]
    def get_feature_names(self):
        return self._features
    
            
v = TupleVectorizer()

### Move the below to input layer

In [127]:
[k for k, v in globals().items() if k.startswith('train_sent')]

['train_sent',
 'train_sent_tokens',
 'train_sent_vectorized',
 'train_sent_vectorizer']

In [155]:
Zp_vectorizer = TupleVectorizer()
Zn_vectorizer = TupleVectorizer()
for row in tqdm_notebook(train_sent_tokens):
    for token in row:
        for field in token[1]:
            Zp_vectorizer.partial_fit(tuple(field[:-1]))
            Zn_vectorizer.partial_fit(tuple([field[0], field[2]]))

A Jupyter Widget

In [160]:
Zn_vectorizer.transform([('name', 1)])

[2]

### LSTM input

In [184]:
train_field_precedence = dict(zip(map(itemgetter(0), sorted(train_infobox_sum.items(), key=itemgetter(1), reverse=True)), range(len(train_infobox_sum))))

In [215]:
def get_lstm_best_field(fields, precedence):
    def get_field_descriptor(field):
        return f'[{field[0]}_{field[1]}]'
    if len(fields) <= 1:
        return fields if not fields else get_field_descriptor(fields[0])
    return get_field_descriptor(fields[np.argmin(map(lambda field: precedence[field], map(itemgetter(0), fields)))])
train_sent_lstm = train_sent_tokens.progress_apply(lambda sent: [get_lstm_best_field(token[1], train_field_precedence) or token[0] for token in sent])

A Jupyter Widget

In [218]:
dump_to_pickle(train_sent_lstm, 'train.sent.lstm.pickle')

dumped


In [220]:
valid_sent_lstm = valid_sent_tokens.progress_apply(lambda sent: [get_lstm_best_field(token[1], train_field_precedence) or token[0] for token in sent])

A Jupyter Widget

In [222]:
dump = True
dump_to_pickle(valid_sent_lstm, 'valid.sent.lstm.pickle')
dump = False

dumped
